In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address       Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.48.4  177.65 KiB  16      100.0%            4e75dd2c-d3f6-4178-9931-d53ebb6cee28  rack1
UN  192.168.48.2  182.74 KiB  16      100.0%            959d43b1-e2ee-476e-80d0-5a55865bd4b2  rack1
UN  192.168.48.3  178.65 KiB  16      100.0%            2f31e030-4347-464c-8997-af77d7838e16  rack1



In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

Traceback (most recent call last):
  File "cassandra/cluster.py", line 3541, in cassandra.cluster.ControlConnection._reconnect_internal
  File "cassandra/cluster.py", line 3563, in cassandra.cluster.ControlConnection._try_connect
  File "cassandra/cluster.py", line 1630, in cassandra.cluster.Cluster.connection_factory
  File "cassandra/connection.py", line 850, in cassandra.connection.Connection.factory
  File "/usr/local/lib/python3.10/dist-packages/cassandra/io/asyncorereactor.py", line 347, in __init__
    self._connect_socket()
  File "cassandra/connection.py", line 917, in cassandra.connection.Connection._connect_socket
OSError: [Errno None] Tried connecting to [('192.168.48.3', 9042)]. Last error: timed out


In [3]:
cass.execute("DROP KEYSPACE IF EXISTS weather")

cass.execute("""
CREATE KEYSPACE IF NOT EXISTS weather
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 3}
""")

cass.execute("""
CREATE TYPE weather.station_record (
    tmin int,
    tmax int
)
""")

cass.execute("""
CREATE TABLE IF NOT EXISTS weather.stations (
    id TEXT,
    name text STATIC,
    date DATE,
    record weather.station_record,
    PRIMARY KEY (id, date)
)
""")

In [4]:
#q1
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [5]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())


:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-615d10dc-31bb-477f-8d03-c3807a7d4a2a;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [6]:

from pyspark.sql.functions import expr
stations_df = spark.read.text("ghcnd-stations.txt")

wisconsin_stations_df = stations_df.select(
    expr("SUBSTRING(value, 1, 11)").alias("id"),
    expr("SUBSTRING(value, 39, 2)").alias("state"),
    expr("SUBSTRING(value, 42, 30)").alias("name")
).filter(expr("state = 'WI'"))

wisconsin_stations = wisconsin_stations_df.collect()
for station in wisconsin_stations:
    cass.execute(
        """
        INSERT INTO weather.stations (id, name)
        VALUES (%s, %s)
        """,
        (station.id, station.name)
    )

In [7]:
#q2
import pandas as pd
pd.DataFrame(cass.execute("""
    SELECT name FROM weather.stations WHERE id = 'US1WIMR0003'
"""))

,name
0,AMBERG 1.3 SW


In [8]:
#q3
pd.DataFrame(cass.execute("""
    SELECT token(id) FROM weather.stations WHERE id = 'USC00470273'
"""))

,system_token_id
0,-9014250178872933741


In [9]:
#q4
import subprocess
row =  pd.DataFrame(cass.execute("""
    SELECT token(id) FROM weather.stations WHERE id = 'USC00470273'
""")).system_token_id[0]
station_token = int(row)

output = subprocess.check_output(['nodetool', 'ring']).decode('utf-8')

tokens = []
for line in output.splitlines():
    if 'Up' in line or 'Down' in line: 
        token = int(line.split()[-1]) 
        tokens.append(token)
tokens.sort()

next_token = None
for token in tokens:
    if token > station_token:
        next_token = token
        break

next_token

-8655705676496366297

In [10]:
!unzip -o records.zip

Archive:  records.zip
  inflating: records.parquet/part-00000-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
 extracting: records.parquet/._SUCCESS.crc  
  inflating: records.parquet/part-00002-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
  inflating: records.parquet/part-00001-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
  inflating: records.parquet/part-00003-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
 extracting: records.parquet/.part-00003-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/_SUCCESS  
 extracting: records.parquet/.part-00000-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/.part-00001-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/.part-00002-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  


In [11]:
from pyspark.sql.functions import col, to_date, first, date_format
df = spark.read.parquet("records.parquet")
df_pivoted = df.groupBy("station", "date").pivot("element",["TMAX","TMIN"]).agg({"value": "first"})
df_final = df_pivoted.withColumn("date", date_format(to_date(col("date"), "yyyyMMdd"), "yyyy-MM-dd"))
df_final.show()

+-----------+----------+------+------+
|    station|      date|  TMAX|  TMIN|
+-----------+----------+------+------+
|USW00014898|2022-01-07| -71.0|-166.0|
|USW00014839|2022-09-24| 194.0| 117.0|
|USW00014839|2022-05-23| 150.0|  83.0|
|USW00014839|2022-10-19|  83.0|  11.0|
|USW00014839|2022-05-29| 261.0| 139.0|
|USR0000WDDG|2022-11-30| -39.0|-106.0|
|USR0000WDDG|2022-01-19| -56.0|-178.0|
|USW00014837|2022-02-22| -38.0| -88.0|
|USR0000WDDG|2022-02-02|-106.0|-150.0|
|USW00014839|2022-04-27|  39.0|   0.0|
|USW00014839|2022-07-08| 222.0| 189.0|
|USW00014839|2022-09-17| 294.0| 200.0|
|USW00014837|2022-06-24| 322.0| 200.0|
|USW00014898|2022-01-29| -60.0|-116.0|
|USW00014839|2022-07-15| 233.0| 156.0|
|USR0000WDDG|2022-02-24| -61.0|-128.0|
|USR0000WDDG|2022-01-30| -33.0|-117.0|
|USR0000WDDG|2022-04-14|  50.0| -17.0|
|USW00014898|2022-07-28| 256.0| 156.0|
|USW00014837|2022-09-06| 256.0| 117.0|
+-----------+----------+------+------+
only showing top 20 rows



In [12]:
import grpc
import station_pb2
import station_pb2_grpc
records = df_final.collect()
channel = grpc.insecure_channel('localhost:5440')
stub = station_pb2_grpc.StationStub(channel)
for i in records:
    request = station_pb2.RecordTempsRequest(
        station=i.station,
        date=str(i.date),
        tmin=int(i.TMIN),
        tmax=int(i.TMAX)
    )
    response = stub.RecordTemps(request)
    

In [13]:
#q5
request = station_pb2.StationMaxRequest(station="USW00014837")
response = stub.StationMax(request)
response.tmax


356

In [14]:
#q6
view = spark.read.format("org.apache.spark.sql.cassandra") \
    .option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3") \
    .option("keyspace", "weather") \
    .option("table", "stations") \
    .load()
view.createOrReplaceTempView("stations")
spark.catalog.listTables()


[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [15]:
#q7
average_diff_df = spark.sql("""
SELECT id, AVG(record.TMAX - record.TMIN) as avg_diff
FROM stations
WHERE record.TMAX IS NOT NULL AND record.Tmin IS NOT NULL
GROUP BY id
""")

average_diff_df.toPandas().set_index('id')['avg_diff'].to_dict()

{'USW00014898': 102.93698630136986,
 'USW00014837': 105.62739726027397,
 'USR0000WDDG': 102.06849315068493,
 'USW00014839': 89.6986301369863}

In [16]:
#q8
!nodetool status

24/04/16 01:25:37 WARN ChannelPool: [s0|p6-db-2/192.168.48.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=432a753b-a5fd-4d90-b6a7-2bd1c023c699, APPLICATION_NAME=Spark-Cassandra-Connector-local-1713230546687}): failed to send request (java.nio.channels.NotYetConnectedException))


Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address       Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.48.4  255.12 KiB  16      100.0%            4e75dd2c-d3f6-4178-9931-d53ebb6cee28  rack1
UN  192.168.48.2  260.57 KiB  16      100.0%            959d43b1-e2ee-476e-80d0-5a55865bd4b2  rack1
UN  192.168.48.3  256.12 KiB  16      100.0%            2f31e030-4347-464c-8997-af77d7838e16  rack1



In [19]:
#q9
response = stub.StationMax(station_pb2.StationMaxRequest(station="USW00014837"))
response.error


'need 3 replicas, but only have 2'

24/04/16 01:26:05 WARN ChannelPool: [s0|p6-db-2/192.168.48.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=432a753b-a5fd-4d90-b6a7-2bd1c023c699, APPLICATION_NAME=Spark-Cassandra-Connector-local-1713230546687}): failed to send request (java.nio.channels.NotYetConnectedException))


In [20]:
#q10
station_id = "USW00014837"
date = "2023-04-15"
tmin = 1919810
tmax = 114514
request = station_pb2.RecordTempsRequest(
    station=station_id,
    date=date,
    tmin=tmin,
    tmax=tmax
)
response = stub.RecordTemps(request)
error_message = response.error

error_message

''